In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import namedtuple

In [3]:
import pandas as pd
import os
from sys import platform
from time import time, sleep
from shutil import copy

In [4]:
from pydgilib_extra import DGILibAverages, LOGGER_OBJECT, LOGGER_CSV, LOGGER_PLOT
from experiment.plotting import *
from experiment.helpers import *

In [5]:
import dgilib_threaded as dgi_t
from atprogram.atprogram import atprogram


In [6]:
import ctypes
try:
    ctypes.windll.shcore.SetProcessDpiAwareness(True)
except:
    pass

Common constants

In [7]:
experiment = "baseline" # or hash
hash_size = 8
mod_precision = 100000
epsilon = 0.5
iterations = 20
duration = 9999
attempt = 1

hash_size_str = ""
if experiment == "hash":
    hash_size_str = str(hash_size)
exprm_file_name_base = experiment + "_" + str(attempt) + hash_size_str + "_" + str(iterations) + "iter"

File locations for experiments

In [8]:
compile_base_dir = os.path.join(os.path.dirname(os.getcwd()),"KalmanARM")
project_dir = os.path.join(compile_base_dir, "STDIO_Redirect_w_TrustZone")

exprm_base_dir = os.path.join(os.getcwd(), "output")
main_file_from = os.path.join(compile_base_dir, "main_" + experiment + ".c")
main_file_to = os.path.join(project_dir, "main.c")

initial_output = os.path.join(exprm_base_dir, "output_arm.csv")

File locations for plotting

In [9]:
exprm_output_base_folder = os.path.join(os.getcwd(),"output", exprm_file_name_base)
original_output_path = os.path.join(os.getcwd(), "output", "original_output.csv")

exprm_output_name = exprm_file_name_base + "_output.csv"
exprm_averages_name = exprm_file_name_base + "_averages.csv"
exprm_output_path = os.path.join(exprm_output_base_folder, exprm_output_name)
exprm_averages_path = os.path.join(exprm_output_base_folder, exprm_averages_name)

DGILib constants

In [10]:
config = {
    "input_acc_file": os.path.join("input", "input_acc.csv"),
    "input_gyro_file": os.path.join("input", "input_gyro.csv"),
    "output_file": exprm_output_path
}

dgilib_config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "plot_pins_method": "highlight",
    "gpio_delay_time": 0.0015,
    "log_folder": exprm_output_base_folder,
    "file_name_base": exprm_file_name_base
}

# dgilib_config_dict_plot = {
#     "loggers": [LOGGER_OBJECT, LOGGER_CSV, LOGGER_PLOT],
#     "plot_pins_method": "highlight"
# }

In [11]:
try:
    os.mkdir(exprm_output_base_folder)
    print("Folder '" + exprm_output_base_folder + "' created.")
except FileExistsError:
    print("WARNING: Folder '" + exprm_output_base_folder + "' already exists.")
    pass

**Edit C/C++ files with hash table information**

In [12]:
hashtable_header_file_path = os.path.join(project_dir, "HashTable", "hashtable.h")
replace_define_value_in_file(hashtable_header_file_path, "HASHSIZE", hash_size)
sleep(1)
replace_define_value_in_file(hashtable_header_file_path, "MOD_PRECISION", mod_precision)
sleep(1)
replace_define_value_in_file(hashtable_header_file_path, "EPSILON", epsilon)

print("File '" + hashtable_header_file_path + "' now contains:")
print_file(hashtable_header_file_path, "#define HASHSIZE")
print_file(hashtable_header_file_path, "#define MOD_PRECISION")
print_file(hashtable_header_file_path, "#define EPSILON")

File 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanARM\STDIO_Redirect_w_TrustZone\HashTable\hashtable.h' now contains:
#define HASHSIZE 8
#define MOD_PRECISION 100000
#define EPSILON 0.5


In [13]:
copy(main_file_from, main_file_to)
print("Copied '"+ main_file_from + "' to '" + main_file_to + "'")

Copied 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanARM\main_baseline.c' to 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanARM\STDIO_Redirect_w_TrustZone\main.c'


**Run KalmanC++**

In [14]:
x86_kalman_dir = os.path.join(os.path.dirname(os.getcwd()),"KalmanC++")
x86_kalman_hash_header = os.path.join(x86_kalman_dir, "HashTable", "hashtable.h")

In [15]:
hashtable_header_file_path = x86_kalman_hash_header
replace_define_value_in_file(hashtable_header_file_path, "HASHSIZE", hash_size)
sleep(1)
replace_define_value_in_file(hashtable_header_file_path, "MOD_PRECISION", mod_precision)
sleep(1)
replace_define_value_in_file(hashtable_header_file_path, "EPSILON", epsilon)

print("File '" + hashtable_header_file_path + "' now contains:")
print_file(hashtable_header_file_path, "#define HASHSIZE")
print_file(hashtable_header_file_path, "#define MOD_PRECISION")
print_file(hashtable_header_file_path, "#define EPSILON")

File 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanC++\HashTable\hashtable.h' now contains:
#define HASHSIZE 8
#define MOD_PRECISION 100000
#define EPSILON 0.5


In [16]:
compile_hash()

In [17]:
run_hash()

Executing ['C:\\Users\\Dragos\\Dropbox\\RISE\\Git\\KalmanC++\\kalman-hash.exe']
Mod precision: 100000
Epsilon: 0.500000
Hash size: 8

HashTable population: 8 / 8
No. of times found: 5
No. of times stored: 94


In [18]:
output_file_from = os.path.join(x86_kalman_dir, "debug.txt")
output_file_to = os.path.join(exprm_output_base_folder, exprm_file_name_base + "_table_stats.txt")
copy(output_file_from, output_file_to)
print("Copied '"+ output_file_from + "' to '" + output_file_to + "'")

Copied 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanC++\debug.txt' to 'C:\Users\Dragos\Dropbox\RISE\Git\PythonCSV\output\baseline_1_20iter\baseline_1_20iter_table_stats.txt'


**Experiment**

In [19]:
if False: atprogram(project_dir, verbose=1)

In [20]:
data, preprocessed_data = dgi_t.measure(duration, iterations, dgilib_config_dict, input_acc_file=config["input_acc_file"], input_gyro_file=config["input_gyro_file"], output_file=config["output_file"], waitForPlot=True)

Reading line 21/21

Written Kalman output to 'C:\Users\Dragos\Dropbox\RISE\Git\PythonCSV\output\baseline_1_20iter\baseline_1_20iter_output.csv'

Receiving thread done!
Measurement thread: Got stop command!
Sending thread done!
Wrote gpio data to C:\Users\Dragos\Dropbox\RISE\Git\PythonCSV\output\baseline_1_20iter\baseline_1_20iter_gpio.csv'
Wrote power data to C:\Users\Dragos\Dropbox\RISE\Git\PythonCSV\output\baseline_1_20iter\baseline_1_20iter_power.csv'
Measurement thread done!


In [21]:
data2 = dgi_t.load_from_csv(exprm_output_base_folder, exprm_file_name_base)

Save averages data

In [22]:
avg1 = DGILibAverages(data = data, average_function="leftpoint", ignore_first_average = False)
avg1.calculate_averages_for_pin(1)
avg1.print_averages_for_pin(1,5)

    1: (   1.06485 s,    1.07509 s) =    0.01024 s         ignored mC
    2: (   1.76961 s,    1.78082 s) =    0.01121 s        0.005686 mC
    3: (   2.36358 s,    2.37503 s) =    0.01145 s        0.005825 mC
    4: (   3.06789 s,    3.07935 s) =    0.01146 s        0.005829 mC
    5: (   3.70594 s,    3.71746 s) =    0.01152 s        0.005868 mC
Average charge per iteration: 5.831852439 uC
Average energy per iteration: 29.159262 uJ
Average time per iteration: 0.011474 ms

Total iterations: 19
Total average current: 0.508258 mA
Total charge: 0.110805 mC
Total energy: 0.554026 mJ
Total time: 0.21801 s

Benchmark time: 0.60762572 s


In [23]:
avg2 = DGILibAverages(data = data2, average_function="leftpoint")
avg2.calculate_averages_for_pin(1)
avg2.print_averages_for_pin(1,5)

    1: (   1.06485 s,    1.07509 s) =    0.01024 s         ignored mC
    2: (   1.76961 s,    1.78082 s) =    0.01121 s        0.005686 mC
    3: (   2.36358 s,    2.37503 s) =    0.01145 s        0.005825 mC
    4: (   3.06789 s,    3.07935 s) =    0.01146 s        0.005829 mC
    5: (   3.70594 s,    3.71746 s) =    0.01152 s        0.005868 mC
Average charge per iteration: 5.831852439 uC
Average energy per iteration: 29.159262 uJ
Average time per iteration: 0.011474 ms

Total iterations: 19
Total average current: 0.508258 mA
Total charge: 0.110805 mC
Total energy: 0.554026 mJ
Total time: 0.21801 s

Benchmark time: 0.61762047 s


In [24]:
avg3 = DGILibAverages(data = data, average_function="pulse")
avg3.calculate_averages_for_pin(1)
avg3.print_averages_for_pin(1,5)

    1: (   1.76961 s,    1.78082 s) =    0.01121 s        0.005721 mC
    2: (   2.36358 s,    2.37503 s) =    0.01145 s        0.005862 mC
    3: (   3.06789 s,    3.07935 s) =    0.01146 s        0.005864 mC
    4: (   3.70594 s,    3.71746 s) =    0.01152 s          0.0059 mC
    5: (   4.52614 s,    4.53767 s) =    0.01153 s        0.005864 mC
Average charge per iteration: 5.866913592 uC
Average energy per iteration: 29.334568 uJ
Average time per iteration: 0.011474 ms

Total iterations: 19
Total average current: 0.511326 mA
Total charge: 0.111471 mC
Total energy: 0.557357 mJ
Total time: 0.218005 s

Benchmark time: 0.07395482 s


In [25]:
avg4 = DGILibAverages(data = data2, average_function="pulse")
avg4.calculate_averages_for_pin(1)
avg4.print_averages_for_pin(1,5)

    1: (   1.76961 s,    1.78082 s) =    0.01121 s        0.005721 mC
    2: (   2.36358 s,    2.37503 s) =    0.01145 s        0.005862 mC
    3: (   3.06789 s,    3.07935 s) =    0.01146 s        0.005864 mC
    4: (   3.70594 s,    3.71746 s) =    0.01152 s          0.0059 mC
    5: (   4.52614 s,    4.53767 s) =    0.01153 s        0.005864 mC
Average charge per iteration: 5.866913592 uC
Average energy per iteration: 29.334568 uJ
Average time per iteration: 0.011474 ms

Total iterations: 19
Total average current: 0.511326 mA
Total charge: 0.111471 mC
Total energy: 0.557357 mJ
Total time: 0.218005 s

Benchmark time: 0.08694863 s


**Read averages from saved csv**

In [26]:
avg2.write_to_csv(exprm_averages_path)

In [27]:
avg_baseline = DGILibAverages()
avg_baseline.read_from_csv(exprm_averages_path)

avg_baseline.initialized = True
print("")
print("-- Baseline --")
avg_baseline.calculate_averages_for_pin(1)

avg_baseline.print_averages_for_pin(1, 5)


-- Baseline --
    1: (   1.06485 s,    1.07509 s) =    0.01024 s         ignored mC
    2: (   1.76961 s,    1.78082 s) =    0.01121 s        0.005686 mC
    3: (   2.36358 s,    2.37503 s) =    0.01145 s        0.005825 mC
    4: (   3.06789 s,    3.07935 s) =    0.01146 s        0.005829 mC
    5: (   3.70594 s,    3.71746 s) =    0.01152 s        0.005868 mC
Average charge per iteration: 5.831852439 uC
Average energy per iteration: 29.159262 uJ
Average time per iteration: 0.011474 ms

Total iterations: 19
Total average current: 0.508258 mA
Total charge: 0.110805 mC
Total energy: 0.554026 mJ
Total time: 0.21801 s

Benchmark time: 0.0 s


**Comparison plot with Intel x86 result**

In [ ]:
f1 = comparison_plot(original_output_path, exprm_output_path, iterations, "Original", "Baseline")
wait_for_plot(f1, True)